In [ ]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [ ]:
#https://github.com/fraunhoferhhi/BerlinV2X
sidelink='/home/bde/Data/BerlinV2X/sidelink_dataframe.parquet'
cellular='/home/bde/Data/BerlinV2X/cellular_dataframe.parquet'
df = pd.read_parquet(cellular)
#df.info(verbose=True)

In [ ]:
DIRECTION_downlink = True
# "direction == 'downlink'
# "direction == 'uplink'
# measured_qos == 'datarate'
# measured_qos == 'delay'
# Filter only for downlink datarate measurements
filtered_data = df.query("direction == 'downlink' & measured_qos == 'datarate'")

# Remove incomplete measurements without datarate
filtered_data = filtered_data.dropna(subset='datarate')

# Train and test split along operators
operator_1 = filtered_data.query("operator == 1")
operator_1_pc_1 = operator_1.query("device == 'pc1'")
operator_1_pc_2 = operator_1.query("device == 'pc2'")
operator_1_pc_3 = operator_1.query("device == 'pc3'")
operator_1_pc_4 = operator_1.query("device == 'pc4'")

print(len(operator_1_pc_1))
print(len(operator_1_pc_2))
print(len(operator_1_pc_3))
print(len(operator_1_pc_4))

operator_2 = filtered_data.query("operator == 2")
operator_2_pc_1 = operator_2.query("device == 'pc1'")
operator_2_pc_2 = operator_2.query("device == 'pc2'")
operator_2_pc_3 = operator_2.query("device == 'pc3'")
operator_2_pc_4 = operator_2.query("device == 'pc4'")

print(len(operator_2_pc_1))
print(len(operator_2_pc_2))
print(len(operator_2_pc_3))
print(len(operator_2_pc_4))

In [ ]:
df_new = operator_2_pc_3[['datarate', 'PCell_RSRP_1','PCell_RSRP_2','PCell_RSRQ_1','PCell_RSRQ_2','PCell_RSSI_1','PCell_RSSI_2','PCell_SNR_1','PCell_SNR_2']]
#df_new = operator_2_pc_2[['datarate', 'SCell_RSRP_1','SCell_RSRP_2','SCell_RSRQ_1','SCell_RSRQ_2','SCell_RSSI_1','SCell_RSSI_2','SCell_SNR_1','SCell_SNR_2']]
df_new = df_new.dropna()
df_new.info()

In [ ]:
df_new[['datarate']].plot(figsize=(9,6))

In [ ]:
df_new['datetime'] = df_new.index

groups=(df_new['datetime'].diff()>pd.Timedelta(seconds=1)).cumsum()+1
max_len = 0
continous_groups = []
for i , group in df_new.groupby(groups):
    if len(group) > 512:
        max_len = len(group)
        continous_groups.append(group)
        print(len(group))
#for group in continous_groups:
    #group.info()

In [ ]:
SCOPE = 0
continous_groups[SCOPE][['datarate']].plot(figsize=(9,6))

In [ ]:
continous_groups[SCOPE][['PCell_RSRP_1', 'PCell_RSRQ_1', 'PCell_RSSI_1', 'PCell_SNR_1']].plot(figsize=(9,6))

In [ ]:
continous_groups[SCOPE][['PCell_RSRP_2', 'PCell_RSRQ_2', 'PCell_RSSI_2', 'PCell_SNR_2']].plot(figsize=(9,6))

In [ ]:
""" df = continous_groups[SCOPE]
df['date'] = df['datetime']
df = df.drop(columns=['operator', 'datetime'])
df.to_parquet(f'data/BERLINV2X/continous_groups__{SCOPE}.parquet.gzip',compression='gzip')
df.to_csv(f'data/BERLINV2X/continous_groups_{SCOPE}.csv', encoding='utf-8', index=False)
df.head() """

In [ ]:
longest = pd.concat(continous_groups)
longest['date'] = longest['datetime']
longest = longest.drop(columns=['datetime'])
longest.to_parquet(f'data/BERLINV2X/consecutive_512_operator_2_pc_3.parquet.gzip',compression='gzip')
longest.to_csv(f'data/BERLINV2X/consecutive_512_operator_2_pc_3.csv', encoding='utf-8', index=False)
longest.info()

In [ ]:
""" qos_column = 'datarate'

downlink_columns = [
     'ping_ms',
     'datarate',
     'PCell_RSRP_max',
     'PCell_RSRQ_max',
     'PCell_RSSI_max',
     'PCell_SNR_1',
     'PCell_SNR_2',
     'PCell_Downlink_Num_RBs',
     'PCell_Downlink_TB_Size',
     'PCell_Downlink_Average_MCS',
     'PCell_Downlink_frequency',
     'PCell_Downlink_bandwidth_MHz',
     'PCell_Cell_Identity',
     'PCell_freq_MHz',
     'SCell_RSRP_max',
     'SCell_RSRQ_max',
     'SCell_RSSI_max',
     'SCell_SNR_1',
     'SCell_SNR_2',
     'SCell_Downlink_Num_RBs',
     'SCell_Downlink_TB_Size',
     'SCell_Downlink_Average_MCS',
     'SCell_Downlink_frequency',
     'SCell_Downlink_bandwidth_MHz',
     'SCell_Cell_Identity',
     'SCell_freq_MHz',
     'Latitude',
     'Longitude',
     'Altitude',
     'speed_kmh',
     'COG',
     'precipIntensity',
     'precipProbability',
     'temperature',
     'apparentTemperature',
     'dewPoint',
     'humidity',
     'pressure',
     'windSpeed',
     'cloudCover',
     'uvIndex',
     'visibility',
     'Traffic Jam Factor']

uplink_columns = [
     'ping_ms',
     'datarate',
     'PCell_RSRP_max',
     'PCell_RSRQ_max',
     'PCell_RSSI_max',
     'PCell_SNR_1',
     'PCell_SNR_2',
     'PCell_Uplink_Num_RBs',
     'PCell_Uplink_TB_Size',
     'PCell_Uplink_Tx_Power_(dBm)',
     'PCell_Uplink_frequency',
     'PCell_Uplink_bandwidth_MHz',
     'PCell_Cell_Identity',
     'PCell_freq_MHz',
     'Latitude',
     'Longitude',
     'Altitude',
     'speed_kmh',
     'COG',
     'precipIntensity',
     'precipProbability',
     'temperature',
     'apparentTemperature',
     'dewPoint',
     'humidity',
     'pressure',
     'windSpeed',
     'cloudCover',
     'uvIndex',
     'visibility',
     'Traffic Jam Factor']

print(f"The BERLINV2X uses {len(downlink_columns)} download features and {len(uplink_columns)} upload features") """

In [ ]:
""" if DIRECTION_downlink:
    # Missing value imputation
    d_1 = train_data_1[downlink_columns].fillna(0)
    d_2 = train_data_4[downlink_columns].fillna(0)
    d_3 = test_data_2[downlink_columns].fillna(0)
    d_4 = test_data_3[downlink_columns].fillna(0)

    downlink = pd.concat([d_1, d_2, d_3, d_4])
    downlink['date'] = downlink.index.tz_localize(None)
    downlink['PCell_Downlink_bandwidth_MHz'] = pd.to_numeric(downlink['PCell_Downlink_bandwidth_MHz'])
    downlink['SCell_Downlink_bandwidth_MHz'] = pd.to_numeric(downlink['SCell_Downlink_bandwidth_MHz'])
    downlink.to_csv(f'data/BERLINV2X/berlin-downlink-delay.csv', encoding='utf-8', index=False)
    downlink.info()
    downlink.head()
    df = downlink """

In [ ]:
""" if not DIRECTION_downlink:
    u_1 = train_data_1[uplink_columns].fillna(0)
    u_2 = train_data_4[uplink_columns].fillna(0)
    u_3 = test_data_2[uplink_columns].fillna(0)
    u_4 = test_data_3[uplink_columns].fillna(0)

    uplink = pd.concat([u_1, u_2, u_3, u_4])
    uplink['date'] = uplink.index.tz_localize(None)
    uplink['PCell_Uplink_bandwidth_MHz'] = pd.to_numeric(uplink['PCell_Uplink_bandwidth_MHz'])
    #uplink['SCell_Uplink_bandwidth_MHz'] = pd.to_numeric(uplink['SCell_Uplink_bandwidth_MHz'])
    uplink.to_csv(f'data/BERLINV2X/berlin-uplink-delay.csv', encoding='utf-8', index=False)
    uplink.info()
    uplink.head()
    df = uplink """

In [ ]:
""" df.head() """

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#fig, ax = plt.subplots(1,3, figsize=(20, 6))
#sns.heatmap(df.corr('pearson'), ax=ax[0], annot=True)
#sns.heatmap(df.corr('spearman'), ax=ax[1], annot=True)
#sns.heatmap(df.corr('kendall'), ax=ax[2], annot=True)

In [ ]:
def multi_corr(df, feature):
    fig, ax = plt.subplots(1,3, figsize=(20, 6))

    # when the Pearson and Spearman values are not much different, 
    # our data tends to not have extreme values (outliers)
    corr1 = df.corr('pearson')[[feature]].sort_values(by=feature, ascending=False)
    corr2 = df.corr('spearman')[[feature]].sort_values(by=feature, ascending=False)
    corr3 = df.corr('kendall')[[feature]].sort_values(by=feature, ascending=False)
    #ordinal correlation (Spearman & Kendall Tau)

    sns.heatmap(corr1, ax=ax[0], annot=True)
    sns.heatmap(corr2, ax=ax[1], annot=True)
    sns.heatmap(corr3, ax=ax[2], annot=True)

In [ ]:
multi_corr(longest, 'datarate')

In [ ]:
#multi_corr(longest, 'ping_ms')